# Using `eval` for code generation

## Running (evaluating / executing) pieces of code


Once we have a piece of code that we have created, e.g. by modifying a previous piece of code or by creating it from scratch, we want to be able to **run** (**evaluate** / **execute**) it, as if we had typed it straight in.

We can run it at global scope using `eval` (short for "evaluate"):

In [17]:
code = :(x = 3)

:(x = 3)

In [5]:
x

LoadError: UndefVarError: x not defined

In [6]:
eval(code)

3

In [7]:
x

3

The code `x = 3` was run and had the desired **side effect**, namely that a global variable `x` was created and bound to the value `3`. 

# Generating "copy-pasted" code

One important use-case for metaprogramming is generating repetitive code, e.g. for making **wrapper types** and for calling external C libraries.

For example, let's think about a simple type that wraps a floating-point number and counts how many times it gets used during a calculation.

In [1]:
mutable struct MyFloat
    value::Float64
    count::Int
end

In [2]:
MyFloat(x) = MyFloat(x, 0)

MyFloat

We can define the sum of two such numbers:

In [7]:
function Base.:+(x::MyFloat, y::MyFloat)
    x.count += 1
    y.count += 1
    
    return MyFloat(x.value + y.value, 0)
end

In [8]:
x = MyFloat(1.0)
y = MyFloat(2.0)

MyFloat(2.0, 0)

In [9]:
z = x + y

MyFloat(3.0, 0)

Now we would like to do the same for `-`, `*` and `/`. We could just copy and paste the code 

Copying and pasting is possible but inefficient and prone to error -- you need to remember to replace the function in both places.

First mock out the code:

In [25]:
op = :+

ex = quote
    Base.$(op)(x::MyFloat, y::MyFloat)
        x.count += 1
        y.count += 1

        return MyFloat( ($op)(x.value, y.value), 0)
    end

Base.remove_linenums!(ex)

quote
    Base.:+(x::MyFloat, y::MyFloat)
    x.count += 1
    y.count += 1
    return MyFloat(x.value + y.value, 0)
end

In [26]:
ex

quote
    Base.:+(x::MyFloat, y::MyFloat)
    x.count += 1
    y.count += 1
    return MyFloat(x.value + y.value, 0)
end

In [19]:
for op in (:+, :-, :*, :/)
    @eval function Base.$(op)(x::MyFloat, y::MyFloat)
        x.count += 1
        y.count += 1

        return MyFloat( ($op)(x.value, y.value), 0)
    end
end

In [21]:
x - y

MyFloat(-1.0, 0)

In [22]:
x

MyFloat(1.0, 4)

# World-age errors

The ability to generate code on the fly is a powerful one, but there is an issue that you may come across: the infamous **world-age errors**; see [this video by Julia Belyakova](https://www.youtube.com/watch?v=d6lTCnhdbqE) for a very good explanation.

This occurs when `eval` is used inside a function, say `f`, to create a new function, say `g`, and then the user tries to call `g` from within `f`. Although it seems like this should be valid, Julia complains:

In [7]:
function f(n)
    
    code = :(g(x) = 2x)
    eval(code)
    
    return g(n)
    
end

f (generic function with 1 method)

In [8]:
f(10)

LoadError: MethodError: no method matching g(::Int64)
The applicable method may be too new: running in world age 29644, while current world is 29645.
[0mClosest candidates are:
[0m  g(::Any) at In[7]:3 (method too new to be called from this world context.)

[Sometimes you will not even get a world-age error; Julia will just complain that there is no such function or method.]

## Solving world-age problems

There are (at least) three possible solutions:

### Make sure to return to "top level" before calling `g`:

In [10]:
function f()
    
    code = :(g(x) = 2x)
    eval(code)
    
end

f (generic function with 2 methods)

In [11]:
f()

g (generic function with 1 method)

In [17]:
@time g(3)

  0.000000 seconds


6

###  Use `Base.invokelatest` 

Use `Base.invokelatest` to insist to Julia that it bypasses this world-age mechanism. This comes with a performance cost:

In [13]:
function f(n)
    
    code = :(g(x) = 2x)
    eval(code)
    
    return Base.@invokelatest g(n)  # or Base.invokelatest(g, n)
    
end

f (generic function with 2 methods)

In [14]:
f(3)

6

In [15]:
@time f(3)

  0.003115 seconds (847 allocations: 59.998 KiB, 85.81% compilation time)


6

###  Replace `eval` with generated functions

Replace `eval` with https://github.com/JuliaStaging/GeneralizedGenerated.jl or https://github.com/SciML/RuntimeGeneratedFunctions.jl instead:

In [21]:
using RuntimeGeneratedFunctions

RuntimeGeneratedFunctions.init(@__MODULE__)

In [25]:
function generate(n)
    
    code = :(g(x) = $n*x)
    @RuntimeGeneratedFunction(code)
        
end

generate (generic function with 1 method)

In [27]:
function f(n)
    g = generate(n)
    
    @time g(3)
    @time g(3)
end

f (generic function with 2 methods)

In [28]:
f(5)

  0.043212 seconds (60.58 k allocations: 3.802 MiB, 99.95% compilation time)
  0.000002 seconds


15

In [23]:
f(3)

6

In [24]:
@time f(3)

  0.008465 seconds (2.28 k allocations: 146.526 KiB, 98.51% compilation time)


6